In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [2]:
target = pd.read_csv('target2',header=None,index_col=0)
train = feather.read_dataframe('best_from_recent_groupped_train')
test = feather.read_dataframe('best_from_recent_groupped_test')

In [3]:
test_cardid = test['card_id']
train = train[[c for c in train if c not in ['card_id', 'first_active_month','last_active_month']]]
test = test[[c for c in train if c not in ['card_id', 'first_active_month','last_active_month']]]

In [4]:
categorical_feats = [x for x in train if re.search('[th]_cat1_[^s]',x)]
categorical_feats += [x for x in train if (re.search('modeplus',x) or re.search('feature_',x))]
categorical_feats = list(set(categorical_feats))

In [5]:
cut = train.shape[0]
train = pd.concat([train,test])
train = pd.get_dummies(train,columns=categorical_feats)
test = train.iloc[cut:,:]
train = train.iloc[:cut,:]

In [ ]:
best_test = 10
best_x_test = np.nan
for x_test in range(1):
    params = {'max_depth': 4, 
              'eta': 0.02, 
              'objective': 'reg:linear',
              'eval_metric': 'rmse',
              'nthreads':2,
              'subsample':0.9,
              'alpha':0.4,
              'tree_method':'gpu_hist',
              'grow_policy':'lossguide',
   #           'grow_policy':'depthwise',
              'max_leaves':50,
              'max_bin':87,
              'seed':77,
              'colsample_bytree':0.9,
              'min_child_weight':19,
              'max_delta_step':13,
              
              

             }

    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))

        dtrain = xgb.DMatrix(train.iloc[trn_idx],label=target.iloc[trn_idx])
        dval = xgb.DMatrix(train.iloc[val_idx],label=target.iloc[val_idx])
        dtest = xgb.DMatrix(test)


        clf = xgb.train(params=params,
              dtrain=dtrain,
              evals=[(dtrain, 'train'),(dval, 'eval')],
              num_boost_round=30000,
              early_stopping_rounds=200,
              verbose_eval=500)


        oof[val_idx] = clf.predict(dval)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = clf.get_fscore().keys()
        fold_importance_df["importance"] = clf.get_fscore().values()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(dtest) / folds.n_splits

    errr = mean_squared_error(oof, target)**0.5
    if errr < best_test:
        best_test = errr
        best_x_test = x_test
    
    print("CV score: {:<8.5f}\n".format(errr))
    print(x_test, '- x_test \n')

fold n°0
[0]	train-rmse:3.93478	eval-rmse:3.98017
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 200 rounds.
[500]	train-rmse:3.56442	eval-rmse:3.67481
[1000]	train-rmse:3.48372	eval-rmse:3.67158
Stopping. Best iteration:
[1101]	train-rmse:3.46794	eval-rmse:3.67055



In [ ]:
sub_df = pd.DataFrame({"card_id":test_cardid.values})
sub_df["target"] = predictions
sub_df.to_csv("submit_with_all_additionsXGBOOST.csv", index=False)

In [ ]:
best_test,best_x_test

In [ ]:
best_test,best_x_test

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

In [ ]:
all_cols = pd.DataFrame(best_features.groupby(['feature'])['importance'].mean()).sort_values(by='importance')
all_cols